# OpenAI API 사전 준비

In [ ]:
# 패키지 설치
!pip install -U openai

In [ ]:
import os
from google.colab import userdata

# 환경 변수 준비 (왼쪽 끝 키 아이콘으로 OPENAI_API_KEY 설정)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

# 클라이언트 준비
client = OpenAI()

# 이미지 생성

In [ ]:
# 추론 실행
response = client.responses.create(
    model="gpt-5",
    input="귀여운 고양이 귀 메이드 그림을 만들어줘",
    tools=[{"type": "image_generation"}],
)

In [ ]:
import base64

# 이미지 파일 저장
image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]
if image_data:
    image_base64 = image_data[0]
    with open("output.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

# 멀티턴 이미지 생성

In [ ]:
# 2턴째 추론 실행
response_fwup = client.responses.create(
    model="gpt-5",
    previous_response_id=response.id, # 응답 ID
    input="카페에서 일하는 모습을 그려줘. 가로로 긴 이미지로.",
    tools=[{"type": "image_generation"}],
)

In [ ]:
# 이미지 파일 저장
image_data_fwup = [
    output.result
    for output in response_fwup.output
    if output.type == "image_generation_call"
]
if image_data_fwup:
    image_base64 = image_data_fwup[0]
    with open("output2.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

# 스트리밍

In [ ]:
# 스트리밍으로 추론 실행
stream = client.responses.create(
    model="gpt-5",
    input="고양이가 햇볕을 쬐고 있는 일러스트를 그려줘",
    stream=True,
    tools=[{"type": "image_generation", "partial_images": 2}],
)

In [ ]:
import base64

# 이미지 파일 저장
for event in stream:
    if event.type == "response.image_generation_call.partial_image":
        idx = event.partial_image_index
        image_base64 = event.partial_image_b64
        image_bytes = base64.b64decode(image_base64)
        with open(f"streaming{idx}.png", "wb") as f:
            f.write(image_bytes)

# 수정 프롬프트

In [ ]:
# 수정 프롬프트 확인
for item in response.output:
    # image_generation_call
    if item.type == "image_generation_call":
        print("==image_generation_call==")
        print(item.revised_prompt)